In [ ]:
from netCDF4 import Dataset
from matplotlib import pyplot as plt
import numpy as np
import datetime
from scipy.interpolate import griddata
from mpl_toolkits.basemap import Basemap
import os

In [ ]:
start_time_era5 = datetime.datetime(1900, 1, 1, 0, 0, 0)
lon_min_era5, lon_max_era5, lat_min_era5, lat_max_era5 = 90, 200, 67, 80

In [ ]:
lat1, lat2 = (90-lat_max_era5)*4, (90-lat_min_era5)*4+1
lon1, lon2 = lon_min_era5*4, lon_max_era5*4+1

In [ ]:
file = '/mnt/hippocamp/DATA/ERA5/w10/era5_uv10m_2020-09.nc'
data = Dataset(file, 'r')

longitude = np.array(data.variables['longitude'][lon_min_era5*4:lon_max_era5*4+1])
latitude = np.array(data.variables['latitude'][(90-lat_max_era5)*4:(90-lat_min_era5)*4+1])
lon_grid, lat_grid = np.meshgrid(longitude, latitude)

In [ ]:
def drawing(u, v, t=None, lon = lon_grid, lat = lat_grid, save=False):
    fig = plt.figure(figsize=(12, 12), dpi=300)
    
    m = Basemap(width=3500000,height=1500000,
                resolution='l',projection='aea',
                lat_1=60, lat_2=65, lon_0=140, lat_0=76)
    m.drawcoastlines()
    m.fillcontinents(color='grey', lake_color='white')
    m.drawparallels(np.arange(-80., 90., 10.), labels=[False, True, True, False])
    m.drawmeridians(np.arange(-180., 180., 20.), labels=[True, True, False, True])
    m.drawmapboundary(fill_color='white')
    
    # Преобразование координат для карты
    x, y = m(lon, lat)
    
    # Вычисление скорости ветра
    speed = np.sqrt(u**2 + v**2)
    
    # Отображение скорости ветра
    speed_plot = m.pcolormesh(x, y, speed, shading='gouraud', cmap='jet', vmin=0)
    cbar=plt.colorbar(speed_plot, label='Wind speed (m/s)', orientation='vertical', shrink=0.30)
    # cbar.set_ticks([0,2,4,6,8,10,12])
    
    # Отображение направления ветра
    m.quiver(x[::4, ::4], y[::4, ::4], u[::4, ::4], v[::4, ::4], scale=30, scale_units='inches')
    # plt.title(f'Wind speed {t.date()}')
    
    plt.show()
    plt.close('all')

In [ ]:
year = 2016
month = 9

In [ ]:
file = f'/mnt/hippocamp/DATA/ERA5/w10/era5_uv10m_{year}-{month:02}.nc'
data = Dataset(file, 'r')

In [ ]:
u10 = np.asarray(data.variables['u10'][:,(90-lat_max_era5)*4:(90-lat_min_era5)*4+1,lon_min_era5*4:lon_max_era5*4+1])
u10.shape

In [ ]:
v10 = np.asarray(data.variables['v10'][:,(90-lat_max_era5)*4:(90-lat_min_era5)*4+1,lon_min_era5*4:lon_max_era5*4+1])
v10.shape

In [ ]:
u10.shape, v10.shape

In [ ]:
list_u10 = np.split(u10, u10.shape[0]/24, axis=0)
array_u10 = np.array([np.mean(a, axis=0) for a in list_u10])

list_v10 = np.split(v10, v10.shape[0]/24, axis=0)
array_v10 = np.array([np.mean(a, axis=0) for a in list_v10])

In [ ]:
u10_mean = np.mean(u10, axis=0)
v10_mean = np.mean(v10, axis=0)
u10_mean.shape, v10_mean.shape

In [ ]:
drawing(u10_mean, v10_mean)

In [ ]:
# lon_min_era5, lon_max_era5, lat_min_era5, lat_max_era5 = 90, 200, 67, 80
border_north = 75
border_south = 72
border_east = 140
border_west = 128

In [ ]:
u10_upd = u10[:,(lat_max_era5-border_north)*4:(lat_max_era5-border_south)*4+1,
              (border_west-lon_min_era5)*4:(border_east-lon_min_era5)*4]
v10_upd = v10[:,(lat_max_era5-border_north)*4:(lat_max_era5-border_south)*4+1,
              (border_west-lon_min_era5)*4:(border_east-lon_min_era5)*4]
u10_upd.shape, v10_upd.shape

In [ ]:
u10_upd.mean(), v10_upd.mean()

In [ ]:
list_u10_upd = np.split(u10_upd, u10_upd.shape[0]/24, axis=0)
array_u10_upd = np.array([np.mean(a, axis=0) for a in list_u10_upd])

list_v10_upd = np.split(v10_upd, v10_upd.shape[0]/24, axis=0)
array_v10_upd = np.array([np.mean(a, axis=0) for a in list_v10_upd])